#### Cartoonizing and Adding Text

In [2]:
from PIL import Image as pili, ImageDraw as pild, ImageFont as pilf, ImageOps as piliops
from fpdf import FPDF
import os
import pandas as pd
import cv2
import numpy as np
from os.path import join

# Function to add a border around the image using ImageOps.expand
def add_border_to_image(image_pil, border_size=(100, 100), border_color='pink'):
    # Ensure the image is in RGB mode before adding the border
    if image_pil.mode in ('RGBA', 'LA'):
        image_pil = image_pil.convert("RGB")
    
    # Add a border around the image using ImageOps.expand
    return piliops.expand(image_pil, border=border_size, fill=border_color)


In [3]:
from PIL import Image
import cv2 
from IPython.display import display

def imgcompress_mem(path_in, k):
    img = cv2.imread(path_in, cv2.IMREAD_UNCHANGED)
    width = int((img.shape[1]) / k)
    height = int((img.shape[0]) / k)
    return cv2.resize(img, (width, height), interpolation=cv2.INTER_AREA)



In [4]:
from PIL import Image, ExifTags
def fix_image_orientation(img):
    try:
        for orientation in ExifTags.TAGS.keys():
            if ExifTags.TAGS[orientation] == 'Orientation':
                break
        exif = img._getexif()
        if exif is not None and orientation in exif:
            if exif[orientation] == 3:
                img = img.rotate(180, expand=True)
            elif exif[orientation] == 6:
                img = img.rotate(270, expand=True)
            elif exif[orientation] == 8:
                img = img.rotate(90, expand=True)
    except (AttributeError, KeyError, IndexError):
        pass  # No EXIF data or orientation info
    return img

In [5]:
def cartoonizebl_mem(path_in, k, blur, line):
    imgr = imgcompress_mem(path_in, k)
    imgc = fix_image_orientation(imgr)
    line_size = line
    blur_value = blur
    
    # Ensure that line_size is odd and greater than 1
    if line_size % 2 == 0:
        line_size += 1
    if line_size <= 1:
        line_size = 3  # Set a default minimum value if it's too small

    gray = cv2.cvtColor(imgc, cv2.COLOR_BGR2GRAY)
    gray_blur = cv2.medianBlur(gray, blur_value)
    bigedges = cv2.adaptiveThreshold(gray_blur, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, line_size, blur_value)
    
    return cv2.bitwise_and(imgc, imgc, mask=bigedges)

In [6]:
# Function to add dialog text to an image
def add_dialog_to_image(image_pil, dialog, font_path="Arial.ttf", font_size=18):
    # Create a drawing context
    draw = pild.Draw(image_pil)
    
    # Load the font
    try:
        font = pilf.truetype(font_path, font_size)
    except IOError:
        font = pilf.load_default()  # Use default font if the specified one is not found
    
    # Define text position (bottom of the image)
    img_width, img_height = image_pil.size
    
    # Get bounding box of the text
    bbox = draw.textbbox((0, 0), dialog, font=font)
    text_w, text_h = bbox[2] - bbox[0], bbox[3] - bbox[1]
    
    # Position the text at the bottom, centered horizontally
    x = (img_width - text_w) // 2
    y = img_height - text_h - 20  # 20 pixels padding from bottom
    
    # Add a semi-transparent background box
    box_padding = 10
    draw.rectangle([(x - box_padding, y - box_padding), 
                    (x + text_w + box_padding, y + text_h + box_padding)], 
                   fill=(0, 0, 0, 180))  # Semi-transparent black
    
    # Add thicker shadow layers for the text (multiple layers of shadows)
    for offset in range(-2, 3):
        draw.text((x + offset, y + offset), dialog, font=font, fill="black")  # Thicker shadow
    
    # Draw the main white text
    draw.text((x, y), dialog, font=font, fill="white")  # Main text

    return image_pil


In [7]:
# Function to process images from a folder based on parameters from CSV, add dialog, and border
def process_images_from_folder(image_folder, output_folder, csv_file, border_size=(60, 60), border_color='white'):
    # Read the parameters from the CSV file
    df = pd.read_csv(csv_file)
    
    # Ensure the output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Loop through each image and corresponding parameters in the CSV file
    for index, row in df.iterrows():
        image_name = row['image']  # The image file name
        k = row['k']  # Compression factor
        blur = row['blur']  # Blur value
        line = row['line']  # Line size
        dialog = row['dialog']  # Dialog text
        
        # Full path to the image
        image_path = join(image_folder, image_name)
        
        # Process the image: compress and cartoonize
        cartoonized_image = cartoonizebl_mem(image_path, k, blur, line)
        
        # Convert the cartoonized image from BGR (OpenCV) to RGB (Pillow)
        cartoonized_image_pil = pili.fromarray(cv2.cvtColor(cartoonized_image, cv2.COLOR_BGR2RGB))
        
        # Add dialog to the image
        cartoonized_image_with_dialog = add_dialog_to_image(cartoonized_image_pil, dialog)
        
        # Add a border around the image using ImageOps.expand
        cartoonized_image_with_border = add_border_to_image(cartoonized_image_with_dialog, border_size=border_size, border_color=border_color)
        
        # Save the cartoonized image with dialog and border to the output folder
        output_image_path = join(output_folder, f"cartoonized_{image_name}")
        cartoonized_image_with_border.save(output_image_path)
        print(f"Processed and saved: {output_image_path}")

# Example usage
image_folder_1 = "Emmy_images_jpeg_pg1"  # Folder containing original images
output_folder_1 = "Emmy_images_output1"  # Folder to save cartoonized images
csv_file_1 = "dialog1.csv"  # CSV file with parameters
# Example usage

process_images_from_folder(image_folder_1, output_folder_1, csv_file_1, border_size=(40, 40), border_color='white')

Processed and saved: Emmy_images_output1\cartoonized_Panel001.jpeg
Processed and saved: Emmy_images_output1\cartoonized_Panel002.jpeg
Processed and saved: Emmy_images_output1\cartoonized_Panel003.jpeg
Processed and saved: Emmy_images_output1\cartoonized_Panel004.jpeg
Processed and saved: Emmy_images_output1\cartoonized_Panel005.jpeg
Processed and saved: Emmy_images_output1\cartoonized_Panel006.jpeg
Processed and saved: Emmy_images_output1\cartoonized_Panel007.jpeg
Processed and saved: Emmy_images_output1\cartoonized_Panel008.jpeg
Processed and saved: Emmy_images_output1\cartoonized_Panel009.jpeg
Processed and saved: Emmy_images_output1\cartoonized_Panel010.jpeg
Processed and saved: Emmy_images_output1\cartoonized_Panel011.jpeg


In [8]:
image_folder_2 = "Emmy_images_jpeg_pg2"  # Folder containing original images
output_folder_2 = "Emmy_images_output2"  # Folder to save cartoonized images
csv_file_2 = "dialog2.csv"  # CSV file with parameters


process_images_from_folder(image_folder_2, output_folder_2, csv_file_2, border_size=(40, 40), border_color='white')

Processed and saved: Emmy_images_output2\cartoonized_Panel001.jpeg
Processed and saved: Emmy_images_output2\cartoonized_Panel002.jpeg
Processed and saved: Emmy_images_output2\cartoonized_Panel003.jpeg
Processed and saved: Emmy_images_output2\cartoonized_Panel004.jpeg
Processed and saved: Emmy_images_output2\cartoonized_Panel005.jpeg
Processed and saved: Emmy_images_output2\cartoonized_Panel006.jpeg
Processed and saved: Emmy_images_output2\cartoonized_Panel007.jpeg
Processed and saved: Emmy_images_output2\cartoonized_Panel008.jpeg
Processed and saved: Emmy_images_output2\cartoonized_Panel009.jpeg
Processed and saved: Emmy_images_output2\cartoonized_Panel010.jpeg


In [9]:
image_folder_3 = "Emmy_images_jpeg_pg3"  # Folder containing original images
output_folder_3 = "Emmy_images_output3"  # Folder to save cartoonized images
csv_file_3 = "dialog3.csv"  # CSV file with parameters


process_images_from_folder(image_folder_3, output_folder_3, csv_file_3, border_size=(40, 40), border_color='white')

Processed and saved: Emmy_images_output3\cartoonized_Panel001.jpeg
Processed and saved: Emmy_images_output3\cartoonized_Panel002.png
Processed and saved: Emmy_images_output3\cartoonized_Panel003.png
Processed and saved: Emmy_images_output3\cartoonized_Panel004.png
Processed and saved: Emmy_images_output3\cartoonized_Panel005.png
Processed and saved: Emmy_images_output3\cartoonized_Panel006.png
Processed and saved: Emmy_images_output3\cartoonized_Panel007.png
Processed and saved: Emmy_images_output3\cartoonized_Panel008.png
Processed and saved: Emmy_images_output3\cartoonized_Panel009.png
Processed and saved: Emmy_images_output3\cartoonized_Panel010.png
Processed and saved: Emmy_images_output3\cartoonized_Panel011.png
Processed and saved: Emmy_images_output3\cartoonized_Panel012.png
Processed and saved: Emmy_images_output3\cartoonized_Panel013.png
Processed and saved: Emmy_images_output3\cartoonized_Panel014.png
Processed and saved: Emmy_images_output3\cartoonized_Panel015.png


In [10]:
image_folder_4 = "Emmy_images_jpeg_pg4"  # Folder containing original images
output_folder_4 = "Emmy_images_output4"  # Folder to save cartoonized images
csv_file_4 = "dialog4.csv"  # CSV file with parameters


process_images_from_folder(image_folder_4, output_folder_4, csv_file_4, border_size=(40, 40), border_color='white')

Processed and saved: Emmy_images_output4\cartoonized_Panel001.png
Processed and saved: Emmy_images_output4\cartoonized_Panel002.png
Processed and saved: Emmy_images_output4\cartoonized_Panel003.png
Processed and saved: Emmy_images_output4\cartoonized_Panel004.png
Processed and saved: Emmy_images_output4\cartoonized_Panel005.png
Processed and saved: Emmy_images_output4\cartoonized_Panel006.png
Processed and saved: Emmy_images_output4\cartoonized_Panel007.png
Processed and saved: Emmy_images_output4\cartoonized_Panel008.png
Processed and saved: Emmy_images_output4\cartoonized_Panel009.png
Processed and saved: Emmy_images_output4\cartoonized_Panel010.png


In [11]:
image_folder_5 = "Emmy_images_jpeg_pg5"  # Folder containing original images
output_folder_5 = "Emmy_images_output5"  # Folder to save cartoonized images
csv_file_5 = "dialog5.csv"  # CSV file with parameters
 
 
process_images_from_folder(image_folder_5, output_folder_5, csv_file_5, border_size=(40, 40), border_color='white')

Processed and saved: Emmy_images_output5\cartoonized_panel001.png
Processed and saved: Emmy_images_output5\cartoonized_panel002.png
Processed and saved: Emmy_images_output5\cartoonized_panel003.png
Processed and saved: Emmy_images_output5\cartoonized_panel004.png
Processed and saved: Emmy_images_output5\cartoonized_panel005.png
Processed and saved: Emmy_images_output5\cartoonized_panel006.png
Processed and saved: Emmy_images_output5\cartoonized_panel007.png
Processed and saved: Emmy_images_output5\cartoonized_panel008.png
Processed and saved: Emmy_images_output5\cartoonized_panel009.png
Processed and saved: Emmy_images_output5\cartoonized_panel010.png


In [12]:
image_folder_6 = "Emmy_images_jpeg_pg6"  # Folder containing original images
output_folder_6 = "Emmy_images_output6"  # Folder to save cartoonized images
csv_file_6 = "dialog6.csv"  # CSV file with parameters


process_images_from_folder(image_folder_6, output_folder_6, csv_file_6, border_size=(40, 40), border_color='white')

Processed and saved: Emmy_images_output6\cartoonized_panel001.png
Processed and saved: Emmy_images_output6\cartoonized_panel002.png
Processed and saved: Emmy_images_output6\cartoonized_panel003.png
Processed and saved: Emmy_images_output6\cartoonized_panel004.png
Processed and saved: Emmy_images_output6\cartoonized_panel005.png
Processed and saved: Emmy_images_output6\cartoonized_panel006.png
Processed and saved: Emmy_images_output6\cartoonized_panel007.png
Processed and saved: Emmy_images_output6\cartoonized_panel008.png
Processed and saved: Emmy_images_output6\cartoonized_panel009.png
Processed and saved: Emmy_images_output6\cartoonized_panel010.png
Processed and saved: Emmy_images_output6\cartoonized_panel011.png
Processed and saved: Emmy_images_output6\cartoonized_panel012.png
Processed and saved: Emmy_images_output6\cartoonized_panel013.png


In [13]:
import os
import numpy as np
from os import listdir
from os.path import isfile, join
from PIL import Image, ExifTags
from fpdf import FPDF
import random

# Function to fix image orientation based on EXIF data
def fix_image_orientation(image_path):
    img = Image.open(image_path)
    try:
        for orientation in ExifTags.TAGS.keys():
            if ExifTags.TAGS[orientation] == 'Orientation':
                break
        exif = img._getexif()
        if exif is not None and orientation in exif:
            if exif[orientation] == 3:
                img = img.rotate(180, expand=True)
            elif exif[orientation] == 6:
                img = img.rotate(270, expand=True)
            elif exif[orientation] == 8:
                img = img.rotate(90, expand=True)
    except (AttributeError, KeyError, IndexError):
        pass  # No EXIF data or orientation info
    return img

# Function to clean up temp files
def cleanup_temp_files(folder):
    # Delete files that start with 'temp_'
    for file in listdir(folder):
        if file.startswith('temp_'):
            file_path = join(folder, file)
            if isfile(file_path):
                os.remove(file_path)
                print(f"Deleted: {file_path}")

# Function to generate a PDF from a folder of images with parameterization
def generate_pdf(image_folder, cover_image_path, output_pdf, columns=(2, 3), page_width=None, page_height=None):
    # Clean up any previous temp files
    cleanup_temp_files(image_folder)
    
    # List all image files, ignoring hidden files like .DS_Store
    onlyfiles = [f for f in listdir(image_folder) if isfile(join(image_folder, f)) and not f.startswith('.')]
    onlyfiles.sort()  # Sort the files for consistency

    if len(onlyfiles) == 0:
        print(f"No images found in folder: {image_folder}")
        return

    print(f"Processing images from folder: {image_folder}")

    # Create PDF
    cover = Image.open(cover_image_path)  # Use the cover image to get the default page size
    page_width = page_width if page_width else cover.size[0]
    page_height = page_height if page_height else cover.size[1]

    pdf = FPDF(unit="pt", format=[page_width, page_height])
    pdf.add_page()

    y_position = 0  # Start placing images from the top

    # Loop to process all images and place them on a single page
    while len(onlyfiles) > 0:
        # Choose the number of columns for the current row
        num_cols = random.choice(columns) if isinstance(columns, tuple) else columns
        num_cols = min(num_cols, len(onlyfiles))  # Ensure we don't pick more columns than images left
        window = onlyfiles[:num_cols]  # Select images for this row
        
        print(f"Processing images for this row: {window}")
        
        # Calculate the image width based on the number of columns
        img_width = page_width // num_cols

        # Calculate remaining height to fit all rows
        remaining_height = page_height - y_position
        img_height = remaining_height // ((len(onlyfiles) + num_cols - 1) // num_cols)  # Approximate rows remaining
        
        if img_height <= 0:
            print("Not enough space to place more images on this page.")
            break  # Stop if there's no more vertical space to place images

        # Display the images horizontally in the same row
        for idx, image in enumerate(window):
            image_path = join(image_folder, image)
            
            # Fix the orientation of the image
            img = fix_image_orientation(image_path)
            
            # Save the fixed image temporarily
            temp_img_path = join(image_folder, f"temp_{image}")
            img.save(temp_img_path)

            # Calculate x_position for each image (based on column)
            x_position = idx * img_width
            
            # Add the image to the PDF at the calculated position
            pdf.image(temp_img_path, x=x_position, y=y_position, w=img_width, h=img_height)
        
        # Move to the next row by adjusting the y_position
        y_position += img_height
        
        # Remove processed images from the list
        onlyfiles = onlyfiles[num_cols:]

    # Output the PDF to a file
    pdf.output(output_pdf, "F")
    print(f"PDF created: {output_pdf}")


# Example usage for multiple folders
folders = [
    ("Emmy_images_output1", 'Test/single_page_collapse1.pdf'),
    ("Emmy_images_output2", 'Test/single_page_collapse2.pdf'),
    ("Emmy_images_output3", 'Test/single_page_collapse3.pdf'),
    ("Emmy_images_output4", 'Test/single_page_collapse4.pdf'),
    ("Emmy_images_output5", 'Test/single_page_collapse5.pdf'),
    ("Emmy_images_output6", 'Test/single_page_collapse6.pdf'),
]

cover_image_path = 'Cover.png'  # The path to the cover image
columns = (2, 3)  # The number of columns per row, can be passed as a tuple for random selection or fixed number
page_width = None  # Optional: Can override the default page width
page_height = None  # Optional: Can override the default page height

# Generate the PDF with the given parameters for each folder
for image_folder, output_pdf in folders:
    generate_pdf(image_folder, cover_image_path, output_pdf, columns, page_width, page_height)


Processing images from folder: Emmy_images_output1
Processing images for this row: ['cartoonized_Panel001.jpeg', 'cartoonized_Panel002.jpeg']
Processing images for this row: ['cartoonized_Panel003.jpeg', 'cartoonized_Panel004.jpeg']
Processing images for this row: ['cartoonized_Panel005.jpeg', 'cartoonized_Panel006.jpeg', 'cartoonized_Panel007.jpeg']
Processing images for this row: ['cartoonized_Panel008.jpeg', 'cartoonized_Panel009.jpeg']
Processing images for this row: ['cartoonized_Panel010.jpeg', 'cartoonized_Panel011.jpeg']
PDF created: Test/single_page_collapse1.pdf
Processing images from folder: Emmy_images_output2
Processing images for this row: ['cartoonized_Panel001.jpeg', 'cartoonized_Panel002.jpeg']
Processing images for this row: ['cartoonized_Panel003.jpeg', 'cartoonized_Panel004.jpeg']
Processing images for this row: ['cartoonized_Panel005.jpeg', 'cartoonized_Panel006.jpeg']
Processing images for this row: ['cartoonized_Panel007.jpeg', 'cartoonized_Panel008.jpeg', 'carto

In [14]:
!pip install PyPDF2


In [17]:
import PyPDF2

# List of PDF files to merge
pdf_files = [
    'Test/Cover.pdf',
    'Test/single_page_collapse1.pdf',
    'Test/single_page_collapse2.pdf',
    'Test/single_page_collapse3.pdf',
    'Test/single_page_collapse4.pdf',
    'Test/single_page_collapse5.pdf',
    'Test/single_page_collapse6.pdf',
    'Test/End.pdf'
]

# Output file for the merged PDF
output_pdf_path = 'Final_output.pdf'

# Create a PDF merger object
pdf_merger = PyPDF2.PdfMerger()

# Loop through the PDF files and append them
for pdf_file in pdf_files:
    pdf_merger.append(pdf_file)

# Write out the merged PDF to a file
pdf_merger.write(output_pdf_path)
pdf_merger.close()

print(f"Merged PDF created: {output_pdf_path}")


Merged PDF created: Final_output.pdf
